In [2]:
#Vamos a importar librerías para calcular TF-IDF
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors, DenseVector
import os
import shutil
import math

# Inicializar Spark
spark = SparkSession.builder \
    .appName("ProyectoSparkie-Similitudes") \
    .master("local[*]") \
    .getOrCreate()

sc = spark.sparkContext

print(" - Spark inicializado - ")

 - Spark inicializado - 


In [3]:
# Aquí se cargan las frecuencias sacadas de Vocabulario luego de aplicar los filtros
freq_path = "../data/processed/frecuencias_rdd"

# Leer el RDD guardado
rdd_freq = sc.textFile(freq_path)

# Función para parsear el formato para que sean tuplas: "(('documento.txt', 'palabra'), frecuencia)"
def parse_freq(line):
    """Convierte string del RDD a tupla Python"""
    # Formato ejemplo: "(('Doc.txt', 'palabra'), 123)", 123 son la frecuencia con la que aparece
    import ast
    parsed = ast.literal_eval(line)
    documento = parsed[0][0]
    palabra = parsed[0][1]
    freq = parsed[1]
    return (documento, palabra, freq)

rdd_parsed = rdd_freq.map(parse_freq)

print("- Primeros 5 registros:")
for item in rdd_parsed.take(5):
    print(item)
    
print(f"\n- Total de pares (documento, palabra): {rdd_parsed.count()}")

- Primeros 5 registros:


('Romeo_and_Juliet_by_William_Shakespeare.txt', 'tragedy', 1)
('Romeo_and_Juliet_by_William_Shakespeare.txt', 'romeo', 316)
('Romeo_and_Juliet_by_William_Shakespeare.txt', 'william', 1)
('Romeo_and_Juliet_by_William_Shakespeare.txt', 'shakespeare', 1)
('Romeo_and_Juliet_by_William_Shakespeare.txt', 'contents', 1)


[Stage 1:=============================>                             (1 + 1) / 2]


- Total de pares (documento, palabra): 821702


In [4]:
# Aquí se calcula el TF

# 1. Calcular total de palabras por documento 
doc_totals = rdd_parsed.map(lambda x: (x[0], x[2])) \
                       .reduceByKey(lambda a, b: a + b)

print("- Total de palabras por documento (primeros 5):")
for item in doc_totals.take(5):
    print(f"  {item[0]}: {item[1]} palabras")

# 2. Calcular TF: freq / total_palabras_doc, la formula que nos dio el profesor de TF = Numero de apariciones del termino N en documento M / Numero de tockens en el doc j
# Para recordar, la estructura es así: ((doc, palabra), freq)
rdd_doc_word_freq = rdd_parsed.map(lambda x: ((x[0], x[1]), x[2]))

# Join con totales: ((doc, palabra), (freq, total))
rdd_with_totals = rdd_doc_word_freq.map(lambda x: (x[0][0], (x[0][1], x[1]))) \
                                   .join(doc_totals) \
                                   .map(lambda x: ((x[0], x[1][0][0]), (x[1][0][1], x[1][1])))

# Calcular con la formula del profe TF = freq / total
rdd_tf = rdd_with_totals.map(lambda x: (x[0], x[1][0] / x[1][1]))

print("\n- TF calculado (primeros 5):")
for item in rdd_tf.take(5):
    print(f"  {item[0]}: TF = {item[1]:.6f}")

- Total de palabras por documento (primeros 5):


  Moby_Dick;_Or,_The_Whale_by_Herman_Melville.txt: 109524 palabras
  Peter_Pan___by_J._M._Barrie.txt: 21561 palabras
  Thus_Spake_Zarathustra__A_Book_for_All_and_None_by_Friedrich_Wilhelm_Nietzsche.txt: 56548 palabras
  A_Christmas_Carol_by_Charles_Dickens.txt: 14192 palabras
  The_Tragical_History_of_Doctor_Faustus_by_Christopher_Marlowe.txt: 10813 palabras

- TF calculado (primeros 5):


[Stage 6:>                                                          (0 + 1) / 1]

  ('Peter_Pan___by_J._M._Barrie.txt', 'barrie'): TF = 0.000093
  ('Peter_Pan___by_J._M._Barrie.txt', 'matthew'): TF = 0.000046
  ('Peter_Pan___by_J._M._Barrie.txt', 'produced'): TF = 0.000046
  ('Peter_Pan___by_J._M._Barrie.txt', '1991'): TF = 0.000046
  ('Peter_Pan___by_J._M._Barrie.txt', 'duncan'): TF = 0.000046


In [5]:
# Calcular ahora el IDF:
# IDF o peso = log(total_documentos / documentos_que_contienen_palabra)

# 1. Contar total de documentos para tenerlo en una variable
total_docs = doc_totals.count()
print(f"- Total de documentos: {total_docs}")

# 2. Contar en cuántos documentos aparece cada palabra
# Estructura así queda: (palabra, 1) -> (palabra, num_docs)
rdd_word_doc_count = rdd_parsed.map(lambda x: (x[1], x[0])) \
                               .distinct() \
                               .map(lambda x: (x[0], 1)) \
                               .reduceByKey(lambda a, b: a + b)

#Comprobar que se hizo bien:
print("\n- Documentos por palabra (primeros 10):")
for item in rdd_word_doc_count.take(10):
    print(f"  '{item[0]}': aparece en {item[1]} documentos")

# Ahora sí calcular IDF
# 3. Calcular IDF
rdd_idf = rdd_word_doc_count.map(lambda x: (x[0], math.log(total_docs / x[1])))

#Verificamos que todo bien:
print("\n- IDF calculado (primeros 10):")
for item in rdd_idf.take(10):
    print(f"  '{item[0]}': IDF = {item[1]:.4f}")


- Total de documentos: 99

- Documentos por palabra (primeros 10):


  'moby': aparece en 2 documentos
  'herman': aparece en 5 documentos
  'melville': aparece en 2 documentos
  'contents': aparece en 87 documentos
  'etymology': aparece en 6 documentos
  'supplied': aparece en 62 documentos
  'sub': aparece en 24 documentos
  'chapter': aparece en 71 documentos
  'carpet': aparece en 49 documentos
  'bag': aparece en 59 documentos

- IDF calculado (primeros 10):


[Stage 14:>                                                         (0 + 1) / 1]

  'moby': IDF = 3.9020
  'herman': IDF = 2.9857
  'melville': IDF = 3.9020
  'contents': IDF = 0.1292
  'etymology': IDF = 2.8034
  'supplied': IDF = 0.4680
  'sub': IDF = 1.4171
  'chapter': IDF = 0.3324
  'carpet': IDF = 0.7033
  'bag': IDF = 0.5176


In [6]:
# Ahora sí calculamos el TF-IDF = TF * IDF

# Preparar TF, lo transformamos: ((doc, palabra), tf) -> (palabra, (doc, tf))
rdd_tf_prep = rdd_tf.map(lambda x: (x[0][1], (x[0][0], x[1])))

# Join con IDF: (palabra, ((doc, tf), idf))
rdd_tf_idf = rdd_tf_prep.join(rdd_idf) \
                        .map(lambda x: ((x[1][0][0], x[0]), x[1][0][1] * x[1][1]))
# Verificamos
print("- TF-IDF calculado (primeros 10):")
for item in rdd_tf_idf.take(10):
    print(f"  Doc: {item[0][0][:40]}... | Palabra: '{item[0][1]}' | TF-IDF: {item[1]:.6f}")

# Guardar TF-IDF en nuestra carpetita de procesados
output_tfidf = "../data/processed/tfidf_rdd"
if os.path.exists(output_tfidf):
    shutil.rmtree(output_tfidf)
rdd_tf_idf.saveAsTextFile(output_tfidf)

print(f"\n- TF-IDF guardado en: {output_tfidf} con rotundo exito")

- TF-IDF calculado (primeros 10):


  Doc: Peter_Pan___by_J._M._Barrie.txt... | Palabra: 'pan' | TF-IDF: 0.000978
  Doc: My_Life_—_Volume_1_by_Richard_Wagner.txt... | Palabra: 'pan' | TF-IDF: 0.000023
  Doc: The_Interesting_Narrative_of_the_Life_of... | Palabra: 'pan' | TF-IDF: 0.000066
  Doc: Alice's_Adventures_in_Wonderland_by_Lewi... | Palabra: 'pan' | TF-IDF: 0.000067
  Doc: Precious_balms.txt... | Palabra: 'pan' | TF-IDF: 0.001809
  Doc: Anna_Karenina_by_graf_Leo_Tolstoy.txt... | Palabra: 'pan' | TF-IDF: 0.000005
  Doc: The_Adventures_of_Roderick_Random_by_T._... | Palabra: 'pan' | TF-IDF: 0.000009
  Doc: Leviathan_by_Thomas_Hobbes.txt... | Palabra: 'pan' | TF-IDF: 0.000026
  Doc: The_Adventures_of_Tom_Sawyer,_Complete_b... | Palabra: 'pan' | TF-IDF: 0.000024
  Doc: The_Aeneid_by_Virgil.txt... | Palabra: 'pan' | TF-IDF: 0.000013



- TF-IDF guardado en: ../data/processed/tfidf_rdd con rotundo exito


In [7]:
# Convertir TF-IDF a vectores por documento para usarlos en pasos posteriores

# 1. Crear vocabulario global con índices
vocab_global = rdd_tf_idf.map(lambda x: x[0][1]).distinct().zipWithIndex()
vocab_dict = vocab_global.collectAsMap()
vocab_size = len(vocab_dict)

#Imprimimos para ver que todo esté bien:
print(f"- Tamaño del vocabulario: {vocab_size} palabras únicas")
print(f"\n- Primeras 10 palabras del vocabulario:")
for word, idx in list(vocab_dict.items())[:10]:
    print(f"  '{word}': índice {idx}")

# 2. Convertir TF-IDF a formato (doc, [(idx, tfidf), ...])
#Formato: (documento, [(índice_palabra, tfidf), ...])
rdd_doc_vectors = rdd_tf_idf.map(lambda x: (x[0][0], (vocab_dict[x[0][1]], x[1]))) \
                            .groupByKey() \
                            .map(lambda x: (x[0], list(x[1])))

print(f"\n- Vectores creados para {rdd_doc_vectors.count()} documentos")
print("\n- Primer documento (primeras 10 dimensiones):")
first_doc = rdd_doc_vectors.first()
print(f"  Documento: {first_doc[0][:50]}...")
print(f"  Vector (primeros 10): {first_doc[1][:10]}")

- Tamaño del vocabulario: 120381 palabras únicas

- Primeras 10 palabras del vocabulario:
  'pan': índice 0
  'wendy': índice 1
  'james': índice 2
  'millennium': índice 3
  'text': índice 4
  'iii': índice 5
  'island': índice 6
  'mermaids': índice 7
  'lagoon': índice 8
  'children': índice 9



- Vectores creados para 99 documentos

- Primer documento (primeras 10 dimensiones):


[Stage 54:>                                                         (0 + 1) / 1]

  Documento: Alice's_Adventures_in_Wonderland_by_Lewis_Carroll....
  Vector (primeros 10): [(0, 6.669931043068998e-05), (3, 0.0001807225347373104), (5, 4.134140948855176e-05), (9, 6.930444449559121e-05), (11, 6.445272348634077e-05), (12, 3.301047795067466e-05), (13, 7.839297565744772e-06), (14, 0.00021513332675391212), (16, 6.685019252815768e-05), (17, 5.656554752382573e-05)]


In [8]:
#Calculamos ahora la similitud por coseno:

def cosine_similarity_sparse(vec1, vec2):
    """
    Calcula similitud coseno entre dos vectores dispersos
    vec1, vec2: lista de tuplas [(índice, valor), ...]
    """
    # Convertir a diccionarios para búsqueda rápida
    dict1 = dict(vec1)
    dict2 = dict(vec2)
    
    # Producto punto
    dot_product = sum(dict1.get(idx, 0) * dict2.get(idx, 0) 
                      for idx in set(dict1.keys()) | set(dict2.keys()))
    
    # Normas (La parte de abajo de la formula [||V||*||W||])
    norm1 = math.sqrt(sum(v**2 for v in dict1.values()))
    norm2 = math.sqrt(sum(v**2 for v in dict2.values()))
    
    if norm1 == 0 or norm2 == 0:
        return 0.0
    
    return dot_product / (norm1 * norm2)

# Broadcast de vectores para comparación eficiente
doc_vectors_list = rdd_doc_vectors.collect()
print(f"- Vectores recolectados: {len(doc_vectors_list)} documentos")

- Vectores recolectados: 99 documentos


In [9]:
# Crear todas las combinaciones de pares de documentos
from itertools import combinations

print("- Calculando similitudes entre todos los pares...")

# Generar pares y calcular similitudes
similarities = []
doc_names = [doc[0] for doc in doc_vectors_list]
total_pairs = len(doc_names) * (len(doc_names) - 1) // 2

print(f"- Total de pares a calcular: {total_pairs}")

#Comparamos cada documento con todos los demás
for i, (doc1, vec1) in enumerate(doc_vectors_list):
    for doc2, vec2 in doc_vectors_list[i+1:]:
        sim = cosine_similarity_sparse(vec1, vec2)
        similarities.append((doc1, doc2, sim))
    
    if (i + 1) % 10 == 0:
        print(f"  Procesados: {i+1}/{len(doc_vectors_list)} documentos...")

# Convertir a RDD para guardarlo después
rdd_similarities = sc.parallelize(similarities)

print(f"\n- Similitudes calculadas: {rdd_similarities.count()} pares")
print("\n- Top 10 pares más similares:")

#Guardamos: (doc1, doc2, similitud)
top_similar = rdd_similarities.sortBy(lambda x: x[2], ascending=False).take(10)
for doc1, doc2, sim in top_similar:
    print(f"  {sim:.4f} | {doc1[:40]}... ↔ {doc2[:40]}...")

- Calculando similitudes entre todos los pares...
- Total de pares a calcular: 4851
  Procesados: 10/99 documentos...
  Procesados: 20/99 documentos...
  Procesados: 30/99 documentos...
  Procesados: 40/99 documentos...
  Procesados: 50/99 documentos...
  Procesados: 60/99 documentos...
  Procesados: 70/99 documentos...
  Procesados: 80/99 documentos...
  Procesados: 90/99 documentos...



- Similitudes calculadas: 4851 pares

- Top 10 pares más similares:


[Stage 66:>                                                         (0 + 1) / 1]

  0.9990 | A_Christmas_Carol_in_Prose;_Being_a_Ghos... ↔ A_Christmas_Carol_by_Charles_Dickens.txt...
  0.9958 | Little_Women;_Or,_Meg,_Jo,_Beth,_and_Amy... ↔ Little_Women_by_Louisa_May_Alcott.txt...
  0.5293 | The_Confessions_of_St._Augustine_by_Bish... ↔ Paradise_Lost_by_John_Milton.txt...
  0.5235 | The_2003_CIA_World_Factbook_by_United_St... ↔ The_2006_CIA_World_Factbook_by_United_St...
  0.4415 | The_Adventures_of_Sherlock_Holmes_by_Art... ↔ The_Hound_of_the_Baskervilles_by_Arthur_...
  0.4249 | The_Confessions_of_St._Augustine_by_Bish... ↔ The_divine_comedy_by_Dante_Alighieri.txt...
  0.4188 | The_divine_comedy_by_Dante_Alighieri.txt... ↔ Paradise_Lost_by_John_Milton.txt...
  0.4090 | A_Study_in_Scarlet_by_Arthur_Conan_Doyle... ↔ The_Adventures_of_Sherlock_Holmes_by_Art...
  0.3596 | The_Iliad_by_Homer.txt... ↔ The_Aeneid_by_Virgil.txt...
  0.3507 | Adventures_of_Huckleberry_Finn_by_Mark_T... ↔ The_Adventures_of_Tom_Sawyer,_Complete_b...


In [10]:
# Guardar matriz ahora sí en nuestra carpeta de procesados
output_sim = "../data/processed/similarity_matrix_rdd"
if os.path.exists(output_sim):
    shutil.rmtree(output_sim)

rdd_similarities.saveAsTextFile(output_sim)
print(f"- Matriz de similitudes guardada en: {output_sim}")

- Matriz de similitudes guardada en: ../data/processed/similarity_matrix_rdd


In [11]:
sc.stop()